# <font color=green>deepBreaks Applications</font>
## Predicting the Phenotype of an Opsin From its Amino Acid Sequence...
</p>

In [ ]:
#All neccessary packages to import for data process steps.
import mysql
#install mysql-connector-python // NOT mysql-connector
import mysql.connector
import argparse
import re
# importing deepBreaks libraries 
from deepBreaks import preprocessing as prp
from deepBreaks import visualization as viz
from deepBreaks import models as ml
import os
import datetime
import warnings
from pycaret.classification import *
import pandas as pd
import matplotlib.pyplot as plt
import random 
import csv

In [ ]:
z = 0
for nums in range(0,100):
    wds = open("wds_ni_fmt.fasta", "r").readlines()
    meta = open("class_ni_meta.tsv", "r",)
    tsv = csv.reader(meta, delimiter="\t")
    wds_meta = open("wds_ni_meta.tsv", "r")
    wds_tsv = csv.reader(wds_meta, delimiter="\t")
    data_columns = ["Seq_Name","Known_Class","Predicted_Class","Lambda_Max","Score","Correct","Test"]
    results_df = pd.DataFrame(columns=data_columns)  
    td = open("trainingdata.fasta", "w")
    md = open("meta_data.tsv", "w")
    ud = open("unseendata.fasta", "w")
    x = 0
    k = 0
    m = 0

    #gives us X random numbers between 1-Y
    randomlist = random.sample(range(1,686), 10)

    for entry in tsv:
        print(entry)
        if m == 0:
            m+=1
        else:    
            #read the index entry for row -> if entry is in ranlist then pass write to the dataframe AND to the unseen data file
            #else you write to the training data and meta file 
            #going to need another if statement that 'passes' if k = 0 or 1, then adds 1
            #Actual sequences start on line '2' because of column names and bovine... 
            if entry[0] == "Bovine":
                data = wds[x]
                td.write(data)
                x+=1
                data = wds[x]
                td.write(data)
                x+=1
                if k == 0:
                    md.write("\tOpsin_Class\n")
                    k+=1
                md.write(entry[0]+"\t"+entry[1]+"\n")
                
            else:
                seq_name = int(entry[0].replace("S",""))
                opc = int(entry[1])

                if seq_name in randomlist:     
                    for max in wds_tsv:
                        if max[0] == entry[0]:
                            lambda_max = max[1]
                            break
                        else:
                            pass
                    #write to dataframe
                    results_df = results_df.append({"Seq_Name":entry[0],"Known_Class":entry[1],"Lambda_Max":lambda_max}, ignore_index = True)
                    print(results_df)
                    #write to the unseen data file
                    data = wds[x].replace("\n","")+"_OPC"+entry[1]+"\n"
                    x+=1
                    data+= wds[x]
                    ud.write(data)
                    x+=1
                    #call the whole dataset data file
                    #search for ">S{k} in file and take that line [add an '_opcX to it] + following line and copy it over to the unseen data file"
                    #write to dictionary?
                
                else:
                    #write to the training data file
                    data = wds[x]
                    td.write(data)
                    x+=1
                    data = wds[x]
                    td.write(data)
                    x+=1
                    if k == 0:
                        md.write("\tOpsin_Class\n")
                        k+=1
                    md.write(entry[0]+"\t"+entry[1]+"\n")
        
    #print(results_df)
    td.close() 
    md.close()
    ud.close()

    # defining user params, file pathes, analysis type

    # path to sequences
    seqFileName = 'trainingdata.fasta' 
    meta_data = 'meta_data.tsv'
    # name of the phenotype
    mt = 'Opsin_Class'

    # type of the sequences
    seq_type = 'amino-acid'
    # type of the analysis if it is a classification model, then we put cl instead of reg
    anaType = 'cl'
    sampleFrac=1

    # making a unique directory for saving the reports of the analysis
    print('direcory preparation')
    dt_label = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    seqFile = seqFileName.split('.')[0]
    report_dir = str(seqFile +'_' + mt + '_' + dt_label)
    os.makedirs(report_dir)

    #%%time
    # importing sequences data
    print('reading fasta file')
    df = prp.read_data(seqFileName, seq_type = seq_type, is_main=True)
    metaData = prp.read_data(meta_data, is_main=False)
    positions = df.shape[1]
    print('Done')
    print('Shape of data is: ', df.shape)

  
    # selecting only the classes with enough number of samples
    df = prp.balanced_classes(dat=df, meta_dat=metaData, feature=mt)

    df.head()

    print('metadata looks like this:')
    metaData.head()

    #%%time
    # taking care of missing data
    print('Shape of data before missing/constant care: ', df.shape)
    df = prp.missing_constant_care(df)
    print('Shape of data after missing/constant care: ', df.shape)
    #%%time

    # taking care of ultra-rare cases
    print('Shape of data before imbalanced care: ', df.shape)
    df = prp.imb_care(dat=df)
    print('Shape of data after imbalanced care: ', df.shape)

    #%%time
    # Use statistical tests to drop redundant features.
    print('number of columns of main data befor: ', df.shape[1])
    df_cleaned = prp.redundant_drop(dat=df, meta_dat=metaData,
                            feature=mt, model_type=anaType,
                            threshold=0.25,
                            report_dir=report_dir)
    print('number of columns of main data after: ', df_cleaned.shape[1])

    #%%time
    print('one-hot encoding the dataset')
    df_cleaned = prp.get_dummies(dat=df_cleaned, drop_first=True)


    #%%time
    print('calculating the distance matrix')
    cr = prp.distance_calc(dat=df_cleaned,
                        dist_method='correlation',
                        report_dir=report_dir)
    print(cr.shape)

    print('The distance matrix looks like this.\n The values are between 0 (exact the same) and 1 (non-related).')
    cr.head()

    #%%time
    print('finding colinear groups')

    dc_df = prp.db_grouped(dat = cr,
                        report_dir=report_dir,
                        threshold=.3,
                        needs_pivot=False)
    print('The result of the last step is a dataframe with two columns,\
    1)feature and 2)group.\nif there are no groups, it will be an empty dataframe')
    dc_df.head()

    #%%time
    print('grouping features')
    dc = prp.group_features(dat=df_cleaned,
                            group_dat=dc_df,
                            report_dir=report_dir)
    #file2 = open('dc_cols.txt', 'a+')
    #training = str(dc)
    #file2.write(training)
    #file2.close()  
                        
    print('dropping correlated features')
    print('Shape of data before collinearity care: ', df_cleaned.shape)
    df_cleaned = prp.cor_remove(df_cleaned, dc)
    print('Shape of data after collinearity care: ', df_cleaned.shape)
    file2 = open('train_cols.txt', 'w')
    training = (df_cleaned.to_string())
    file2.write(training)
    file2.close()

    #USE FOR TRAIN COLS! It's the correct format. Import into excel. 

    training_col_list = df_cleaned.columns.tolist()
    #merge with meta data
    df = df.merge(metaData[mt], left_index=True, right_index=True)
    df_cleaned = df_cleaned.merge(metaData[mt], left_index=True, right_index=True)

    #file2 = open('testing_cols.txt', 'a+')
    #raw_training = df.to_string()
    #training = df_cleaned.to_string()
    #file2.write(raw_training)
    #file2.write(training)
    #file2.close()

    #%%time
    models_to_select = 3 # number of top models to select
    top_models, train_cols, model_names = ml.fit_models(dat = df_cleaned,
                                                        meta_var=mt,
                                                        model_type=anaType, 
                                                        models_to_select=models_to_select,
                                                        report_dir=report_dir)

    for i in range(models_to_select):
        # calculate the featre importances
        imp = ml.fimp_single(trained_model=top_models[i], model_name=model_names[i],
                            train_cols=train_cols, grouped_features=dc,
                            n_positions=positions, report_dir=report_dir)
        # visualize the lollipop plot for features based on each model
        viz.dp_plot(dat = imp, model_name= model_names[i],imp_col='standard_value', report_dir=report_dir)
        
        # visualize the boxplots for features based on each model
        viz.plot_imp_model(dat=df, trained_model=top_models[i],
                        model_name=model_names[i],
                        train_cols=train_cols, grouped_features=dc,
                        meta_var=mt, n_positions=positions,
                        model_type=anaType, report_dir=report_dir)

        #file2 = open('interpret_cols.txt', 'a+')
        #training = str(df.to_string)
        #file2.write(training)
        #file2.close()
        
    # merging the results for all the top models
    mean_imp = ml.fimp_top_models(trained_models=top_models, model_names=model_names, 
                                train_cols=train_cols,grouped_features=dc,
                                n_positions=positions,report_dir=report_dir)

    # lollipop plot for the merged results
    viz.dp_plot(dat=mean_imp,
                model_name= 'mean',
                imp_col='mean_imp', 
                report_dir=report_dir)
                
    # visualizing top positions 
    #viz.plot_imp_all(trained_models=top_models, dat=df, train_cols=train_cols,
                    #grouped_features=dc, meta_var=mt, model_type=anaType,
                    #n_positions=positions, report_dir=report_dir)

    models = pd.read_csv(report_dir+ '/models_summary.csv', index_col=0)


    fig, ax = plt.subplots(figsize=(4, 2.5), dpi=350)
    fig = models.iloc[:6,].plot(x="Model", y=["Accuracy", "AUC", "F1"],
                            kind="barh",
                            color=['#648FFF', '#DC267F', '#FFB000'],
                            ax=ax,
                            ylim=(0,1))
    ax.legend(bbox_to_anchor=(0.85, -.1), fontsize = 6, ncol=3)
    ax.set_title('Model performances', fontsize=10)
    ax.set_xlabel('')
    plt.xticks(fontsize=6)
    plt.ylabel('')
    plt.xlim(.98,1)
    plt.yticks(fontsize=8)
    plt.tight_layout()
    plt.savefig(str(report_dir + '/model_performances.pdf'),
                    bbox_inches='tight')

    pos = int(0)
    val = int(0)
    data = list()
    seq_dict = dict()
    unseenData = 'unseendata.fasta' 
    seq_type2 = 'amino-acid'
    #Following creates a dataframe of aa sequences and their positions as collumns.
    df = prp.read_data(unseenData, seq_type = seq_type2, is_main=True)
    df.head()
    #Turning sequence names into a list for indexing
    seq_name = df.index.to_list()
    print(seq_name)
    #for every sequence in the list of sequences the following function will occur. 
    for seq in seq_name:
        data = list()
        for col in training_col_list:
            #take the str of the column name and split at the '_'
            split = str(col).split('_')
            #declare a temp position variable to index df 
            position = str(split[0])
            print(position)
            #declare the aa we will compare the df index to
            aa = str(split[1])
            print(aa)
            count = len(aa)
            #index the df column corresponding to the 'position' variable
            s = df[position]
            #assign value based on whether the aa at some position matches the training_col aa 
            if (s[seq_name[pos]] == aa):
                val = int(1)
            
            elif (count > 1):
                for n in range(count):
                    amino = aa[n]
                    
                    if (s[seq_name[pos]] == amino):
                        val = int(1)
                        break
                    else:
                        val = int(0)   

            else:
                val = int(0)

            data.append(val)
            #print([type(i) for i in data])
            
        seq_dict.update({seq_name[pos] : data})
        pos+=1
    print(seq_dict)
    df_for_testing = pd.DataFrame(data = seq_dict.values() , columns = training_col_list, index = seq_dict.keys(), dtype = int)
    print(df_for_testing.head())

    file3 = open('fmt_data_cols.txt', 'w')
    training = str(df_for_testing.to_string())
    file3.write(training)
    file3.close()

    #interact//model for classification
    # save pipeline
    save_model(top_models[0], 'YOUR_MODEL_NAME')
    # load pipeline
    YOUR_MODEL_NAME = load_model('YOUR_MODEL_NAME')

    #interpret_model(top_models[0])
 
    #save prediction as dataframe
    prediction = predict_model(YOUR_MODEL_NAME, data = df_for_testing)
    print(prediction)
    for i in range(0,10):
        class_prediction = prediction.Label[i]
        cl = int(class_prediction)
        #score = prediction.Score[i]
        #call entry[i] from the results dataframe with something like...
        #results_df.loc[i] = {"Predicted_Class":cp,"Score":score}
        results_df.Predicted_Class[i] = class_prediction
        #results_df.Score[i] = score 
        known = int(results_df.Known_Class[i])
        if cl == known:
            results_df.Correct[i] = 1
        else:
            results_df.Correct[i] = 0
        results_df.Test = nums
        #add a correct/incorrect? == 0 or 1
        
    if z == 0:
        results_df.to_csv(path_or_buf="classifier_test_results.csv",index = False,mode="a")
        z+=1
    else: 
        results_df.to_csv(path_or_buf="classifier_test_results.csv",index = False,mode="a",header = False)

    meta.close()
    wds_meta.close
